In [1]:
import pandas as pd
import numpy as np
import math
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

import platform
from os import listdir
from os.path import isfile, join
from glob import glob
from pathlib import Path
import sys
import os
import copy
import traceback
from pathlib import Path


import matplotlib.pyplot as plt

import SMOTE
import feature_selector
import DE
import CFS
import birch
import metrics.abcd

import metrices
import measures

import sys
import traceback
import warnings
warnings.filterwarnings("ignore")

In [3]:
data_source1 = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp1/1'
if platform.system() == 'Darwin' or platform.system() == 'Linux':
    _dir = data_source1 + '/'
else:
    _dir = data_source1 + '\\'

clusters = [(join(_dir, f)) for f in listdir(_dir) if Path(join(_dir, f)).is_dir()]

In [4]:
#get stats test data
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
size = 0
for cluster in clusters:
    df = pd.read_csv(cluster + '/1385_LR_bellwether_pf.csv')
    size += df.shape[1]
    counts = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values[1:]
        value = sum(i <= 0.33 for i in j)
        counts.append(value)
    df['>=0.6'] = counts
    precentage = max(counts)/df.shape[0]
    score = score + df.iloc[counts.index(max(counts))].values[1:].tolist()
    count += max(counts)
with open('data/1385/exp1/level1.txt', 'w') as f:
    for item in score:
        f.write("%s " % item)

In [ ]:
#get stats test data for finding bellwether
count = 0
count_not = 0
count_yes = 0
precentage = 0
score = []
size = 0
for cluster in clusters[0:1]:
    print(cluster)
    df = pd.read_csv(cluster + '/1385_LR_bellwether_f1.csv')
    size += df.shape[1]
    counts = []
    for row in range(df.shape[0]):
        j = df.iloc[row].values
        counts.append(j)
    with open('data/1385/exp1/level1.txt', 'w') as f:
        for project in counts:
            f.write("%s \n" % project[0])
            for item in project[1:]:
                f.write("%s " % item)
            f.write("\n\n")

In [ ]:
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp3/fold_'
total = []
for i in range(10):
    data_path = path + str(i)
    df = pd.read_csv(data_path + '/bellwether_f1.csv')
    df.columns = ['project','bellwether_level2','self_2','bellwether_level1','self_1','bellwether_level0','self_0']
    df.drop(labels = ['self_1','self_0'],axis = 1, inplace = True)
    for col in df.columns:
        if col == 'project':
            continue
        total.append(np.array(df[col].values.tolist()))
        total = np.concatenate(total).ravel()
        with open('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp3/stats/f1_fold_' + str(i) + '.txt', 'w') as f:
            for item in total:
                f.write("%s " % item.split('[')[1].rsplit(']')[0])



In [5]:
#Latest 
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf/exp_cdom1/fold_'
goals = ['precision','recall','pf','ifa','pci_20']#,'g','precision','recall','pf']

for goal in goals:
    level = {}
    for col in ['self_0','BUBBLE_level2','BUBBLE_level1','BUBBLE_level0','bellwether0']:
        total = []
        for i in range(20):
            data_path = path + str(i)
            df = pd.read_csv(data_path + '/bellwether_' + goal + '.csv')
            df = df.dropna()
            df.columns = ['project','BUBBLE_level2','bellwether0','self_2','BUBBLE_level1','bellwether1','self_1','BUBBLE_level0','bellwether2','self_0']
            df.drop(labels = ['self_2','self_1','bellwether1','bellwether2'],axis = 1, inplace = True)
            total.append(np.array(df[col].values.tolist()))
        total = np.concatenate(total).ravel()
        level[col] = total
    with open('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf/exp_cdom1/stats/' + goal + '.txt', 'w') as f:
        for key in level.keys():
            f.write("%s \n" % key)
            for item in level[key]:
                try:
                    if item == 'nan':
                        print(item)
                        continue
                    f.write("%s " % item)
                except:
                    print('error')
                    continue
            f.write("\n\n")



In [4]:
#Latest with global
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf_v2/exp_cdom2/fold_'
goals = ['precision','recall','pf','ifa','pci_20']#,'g','precision','recall','pf']

for goal in goals:
    level = {}
    for col in ['self_0','BUBBLE_level2','BUBBLE_level1','BUBBLE_level0','bellwether0','global2','random2']:
        total = []
        for i in range(20):
            data_path = path + str(i)
            df = pd.read_csv(data_path + '/bellwether_' + goal + '.csv')
            df = df.dropna()
            df.columns = ['project','BUBBLE_level2','bellwether0','self_2','global2','random2','BUBBLE_level1','bellwether1','self_1','global1','random1','BUBBLE_level0','bellwether2','self_0','global0','random0']
            df.drop(labels = ['self_2','self_1','bellwether1','bellwether2'],axis = 1, inplace = True)
            total.append(np.array(df[col].values.tolist()))
        total = np.concatenate(total).ravel()
        level[col] = total
    with open('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/new_bellwether_pre_re_pf_v2/exp_cdom2/stats/' + goal + '.txt', 'w') as f:
        for key in level.keys():
            f.write("%s \n" % key)
            for item in level[key]:
                try:
                    if item == 'nan':
                        print(item)
                        continue
                    f.write("%s " % item)
                except:
                    print('error')
                    continue
            f.write("\n\n")




In [ ]:
path = '/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp5/fold_'
goals = ['precision','recall','pf','ifa','pci_20']#,'g','precision','recall','pf']

for goal in goals:
    level = {}
    total = []
    for i in range(20):
        data_path = path + str(i)
        df = pd.read_csv(data_path + '/results/' + goal + '.csv')
        df = df.dropna()
        total.append(df[goal].values.tolist())
    total = np.concatenate(total).ravel()
    with open('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp5/stats/' + goal + '.txt', 'w') as f:
        for item in total:
            try:
                if item == 'nan':
                    print(item)
                    continue
                f.write("%s " % item)
            except:
                print('error')
                continue
        f.write("\n\n")


In [12]:
df = pd.read_csv('/Users/suvodeepmajumder/Documents/AI4SE/bellwether_comminity/src/data/1385/exp1/self.csv')

In [32]:
with open('data/1385/exp1/level2.txt', 'w') as f:
    for item in df.f1.values.tolist():
        f.write("%s " % item)